In [1]:
import tensorflow as tf
from tensorflow import keras

from time import time
from tensorflow.python.keras.callbacks import TensorBoard

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import pickle
from sklearn import metrics

print(tf.__version__)

2.0.0-beta0


In [2]:
PATH = os.getcwd()

In [4]:
X_train = pd.read_csv(PATH + "/Train/X_train.csv", header=0)
Y_train = pd.read_csv(PATH + "/Train/Y_train.csv", header=0)

X_test = pd.read_csv(PATH + "/Test/X_test.csv", header=0)
Y_test = pd.read_csv(PATH + "/Test/Y_test.csv", header=0)

In [154]:
def ensemble_prediction(predictions):
    after = []
    j = 0
    for i in range(0,len(X_test)):
        if np.array_equal(predictions[0][i],predictions[1][i]):
            mo = predictions[0][i]
            after.append(list(mo))
            continue
        if np.array_equal(predictions[0][i],predictions[2][i]):
            mo = predictions[0][i]
            after.append(list(mo))
            continue
        if np.array_equal(predictions[1][i],predictions[2][i]):
            mo = predictions[1][i]
            after.append(list(mo))
            continue
        j = j+1
        after.append(list(predictions[1][i]))
    out = np.asarray(after)
    return out, j

In [130]:
with open(PATH + '/Random_Forest/rf3.cpickle', 'rb') as f:
    rf = pickle.load(f)

with open(PATH + '/Decision_Tree/dt3.cpickle', 'rb') as f:
    dt = pickle.load(f)

In [131]:
#model1 = keras.models.load_model(PATH + '/NN_models/3layers_mlp.h5')
model1 = dt
model2 = keras.models.load_model(PATH + '/NN_models/5layers_residual.h5')
#model3 = keras.models.load_model(PATH + '/NN_models/5layers_residual2.h5')
model3 = rf

pred1 = model1.predict(X_test)
pred1 = tf.one_hot(tf.argmax(pred1, axis=1), depth = 3)
pred2 = model2.predict(X_test)
pred2 = tf.one_hot(tf.argmax(pred2, axis=1), depth = 3)
pred3 = model3.predict(X_test)
pred3 = tf.one_hot(tf.argmax(pred3, axis=1), depth = 3)

predictions = [pred1, pred2, pred3]
print(pred1.shape)

(3950, 3)


In [133]:
print(pred2.shape)
print(pred3.shape)
print("Accuracy:",metrics.accuracy_score(Y_test, pred1))

(3950, 3)
(3950, 3)
Accuracy: 0.6820253164556962


In [157]:
Y_ensemble,i = ensemble_prediction(predictions)
print(Y_ensemble.shape)
print(i)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_ensemble))
#2810, 314, 536, 290

(3950, 3)
290
Accuracy: 0.759493670886076


In [141]:
print("Accuracy:",metrics.accuracy_score(Y_test, predictions[2]))

Accuracy: 0.7020253164556962


In [138]:
# DT: Accuracy: 0.6820253164556962
# ResNet: Accuracy: 0.7612658227848101
# RF: Accuracy: 0.7020253164556962

# Ensemble: Accuracy: 0.759493670886076, even worse


tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]], shape=(3950, 3), dtype=float32)


Accuracy: 0.7620253164556962


[[2.05294834e-03 8.08481038e-01 1.89466029e-01]
 [3.42200790e-03 6.84177026e-01 3.12400967e-01]
 [9.97771293e-01 1.68558769e-03 5.43156755e-04]
 ...
 [9.99521524e-01 4.74258297e-04 4.20277320e-06]
 [9.99777168e-01 2.14581552e-04 8.21644880e-06]
 [2.57617056e-01 1.96539626e-01 4.58432958e-02]]
